In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import LeaveOneOut
from imblearn.over_sampling import SMOTE, ADASYN
#from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import random

In [2]:
class RemoveCorrelationTransformer2(BaseEstimator, TransformerMixin):
    def __init__(self, correlation_threshold=0.7):
        self.correlation_threshold = correlation_threshold


    def fit(self, X, Y=None):
        df = pd.DataFrame(X)
        df_corr = df.corr(method='pearson', min_periods=1)
        df_not_correlated = ~(df_corr.mask(
            np.tril(np.ones([len(df_corr)] * 2, dtype=bool))).abs() > self.correlation_threshold).any()
        self.un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
        return self

    def transform(self, X, Y=None):
        df = pd.DataFrame(X)
        df = df[self.un_corr_idx]
        return df.values

In [3]:

class RemoveCorrelationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, correlation_threshold=0.7, pca_components_ratio=3):
        self.correlation_threshold = correlation_threshold
        self.pca_components_ratio = pca_components_ratio


    def fit(self, X, Y=None):
        df = pd.DataFrame(X)
        df_corr = df.corr(method='pearson')
        df_corr = df_corr - np.eye(df.shape[1])
        outliares_corr = df_corr[np.abs(df_corr) > self.correlation_threshold]
        self.outliares_corr = outliares_corr.dropna(axis=1, how='all')

        correlated_df = df[self.outliares_corr.columns]

        n_components = len(self.outliares_corr.columns) // self.pca_components_ratio
        pca = PCA(n_components=n_components)

        correlated_df = pca.fit_transform(correlated_df)
        self.correlated_df = pd.DataFrame(correlated_df, columns=["pca_{}".format(i) for i in range(n_components)])

        return self

    def transform(self, X, Y=None):
        df = pd.DataFrame(X)
        df = df.drop((self.outliares_corr.columns), axis=1)
        df = df.join(self.correlated_df)
        return df

In [4]:

class RemoveMissingFeaturesTransformer(BaseEstimator, TransformerMixin):

    def fit(self, X, Y=None):
        self.is_missing = X.isnull().values.any(axis=0)
        return self

    def transform(self, X, Y=None):
        copy_x = pd.DataFrame(X)
        self.is_missing += copy_x.isnull().values.any(axis=0)

        copy_x = copy_x.iloc[:, ~self.is_missing]

        return copy_x.values


In [5]:
def refactor_labels(df):
    return df.replace({'low': 0 ,'high': 1, 'clinical': 1 })


def get_data(file_name, LSAS_threshold=None):
    group_column = 'group'
    sub_num_col = 'Subject_Number'
    lsas_col = 'LSAS'
    df = pd.read_excel(file_name, sheet_name='Sheet1')
    if LSAS_threshold is None:
        X = df.drop([group_column, sub_num_col, lsas_col], 1)
        Y = refactor_labels(df[group_column])
        return X, Y
    else:
        X = df.drop([group_column], 1)
        Y = pd.Series(np.where(X[lsas_col] > LSAS_threshold, 1, 0))
        X = X.drop([sub_num_col, lsas_col], 1)
        return X, Y


In [6]:
features_lists = [
['average_fixation_length_White_Space', 'avg_of_sum_fixation_length_Disgusted', 'avg_of_amount_fixation_Neutral', 'average_fixation_length_Neutral', 'STD_fixation_length_White_Space', 'STD_fixation_length_All', 'STD_pupil_size_Neutral']
]

## get training data

In [7]:
file_name = "training_set_100.xlsx"

In [8]:
X_train, y_train = get_data(file_name, LSAS_threshold = 50)

## shuffle

In [9]:
random.seed(217828)

In [10]:
X_train.columns

Index(['avg_of_sum_fixation_length_Disgusted',
       'avg_of_sum_fixation_length_Neutral',
       'avg_of_sum_fixation_length_White_Space',
       'average_fixation_length_Disgusted', 'average_fixation_length_Neutral',
       'average_fixation_length_White_Space',
       'avg_of_amount_fixation_Disgusted', 'avg_of_amount_fixation_Neutral',
       'avg_of_amount_fixation_White_Space', 'STD_fixation_length_Disgusted',
       'STD_fixation_length_Neutral', 'STD_fixation_length_White_Space',
       'STD_fixation_length_All', 'Ratio D/D+N', 'Ratio N/D+N',
       'var_ratio_D_DN', 'average_pupil_size_Disgusted',
       'average_pupil_size_Neutral', 'average_pupil_size_White_Space',
       'average_pupil_size_All', 'STD_pupil_size_Disgusted',
       'STD_pupil_size_Neutral', 'STD_pupil_size_White_Space',
       'STD_pupil_size_All', 'mean_different_AOI_per_trial'],
      dtype='object')

In [11]:
columns_shuffled = list(features_lists[0])

In [12]:
random.shuffle(columns_shuffled)
X_train = X_train[columns_shuffled]

In [13]:
len(X_train.columns)

7

In [14]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, stratify=y_train)


## EDA 

In [15]:
X_train.shape

(100, 7)

## get test data

In [16]:
file_name = "full_test_set.xlsx"
df = pd.read_excel(file_name, sheet_name='Sheet1')
X_test = df.drop('Subject_Number', axis = 1)

## training pipeline
 


In [17]:
pipe =  Pipeline([
    ('classifier', XGBClassifier())])

In [18]:
params_grid = [
    {
        'classifier__min_child_weight': [1],
        'classifier__gamma': np.arange(0.2,1, 0.3),
        'classifier__subsample': [0.99, 0.9, 0.8],
        'classifier__colsample_bytree': [0.7, 0.9],
        'classifier__max_depth': [3, 4],
        'classifier__reg_alpha' : [0.5, 0.8],
        'classifier__reg_lambda' :  [0.2, 0.5],
        'classifier__learning_rate': [0.05, 0.1],
        'classifier__n_estimators': [ 75, 95, 150]}] 

## grid search

In [19]:
cv = LeaveOneOut()
gs = GridSearchCV(pipe, params_grid, cv=cv, scoring='accuracy')
gs.fit(X_train_2, y_train_2)

GridSearchCV(cv=LeaveOneOut(), error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('classifier', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'classifier__min_child_weight': [1], 'classifier__gamma': array([0.2, 0.5, 0.8]), 'classifier__subsample': [0.99, 0.9, 0.8], 'classifier__colsample_bytree': [0.7, 0.9], 'classifier__max_depth': [3, 4], 'classifier__reg_alpha': [0.5, 0.8], 'classifier__reg_lambda': [0.2, 0.5], 'classifier__learning_rate': [0.05, 0.1], 'classifier__n_estimators': [75, 95, 150]}],
       pre_dispatch='2*n_jobs', refit=True, r

In [20]:
gs.best_params_

{'classifier__colsample_bytree': 0.7,
 'classifier__gamma': 0.2,
 'classifier__learning_rate': 0.05,
 'classifier__max_depth': 4,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 75,
 'classifier__reg_alpha': 0.8,
 'classifier__reg_lambda': 0.5,
 'classifier__subsample': 0.9}

In [21]:
gs.best_score_

0.7

## validate grid search score LOO

In [22]:
results = []
k = 100
for i in range(k):
    loo = LeaveOneOut()
    score = cross_val_score(gs.best_estimator_, X_train_2, y_train_2, cv=loo)
    results.append(score.mean())
    print(i)
sum(results)/k

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


0.7000000000000013

## validate grid search score  10 folds

In [23]:
results = []
k = 100
for i in range(k):
    cv = StratifiedKFold(10)
    score = cross_val_score(gs.best_estimator_, X_train_2, y_train_2, cv=cv)
    results.append(score.mean())
    print(i)
sum(results)/k

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


0.6608333333333344

## holdout

In [24]:
model = gs.best_estimator_.fit(X_train_2, y_train_2)

In [25]:
y_pred = model.predict(X_test_2)

In [26]:
accuracy_score(y_pred, y_test_2)

0.6

## scores on full set LOO

In [27]:
results = []
k = 100
for i in range(k):
    loo = LeaveOneOut()
    score = cross_val_score(gs.best_estimator_, X_train, y_train, cv=loo)
    results.append(score.mean())
    print(i)
sum(results)/k

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


0.6300000000000011

## scores on full set 10 folds

In [28]:
results = []
k = 100
for i in range(k):
    cv = StratifiedKFold(10)
    score = cross_val_score(gs.best_estimator_, X_train, y_train, cv=cv)
    results.append(score.mean())
    print(i)
sum(results)/k

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


0.6143434343434336